# Classification on `emnist`

## 1. Create `Readme.md` to document your work

Explain your choices, process, and outcomes.

## 2. Classify ~~all symbols~~ letters a -> g

### Subset the data

Select only the lowercase letters (a, b, ..., g) for classification

### Choose a model

Your choice of model! Choose wisely...

### Train away!

Is do you need to tune any parameters? Is the model expecting data in a different format?

### Evaluate the model

Evaluate the models on the test set, analyze the confusion matrix to see where the model performs well and where it struggles.

### Investigate subsets

On which classes does the model perform well? Poorly? Evaluate again, excluding easily confused symbols (such as 'O' and '0').

### Improve performance

Brainstorm for improving the performance. This could include trying different architectures, adding more layers, changing the loss function, or using data augmentation techniques.

## 3. Model showdown: upper vs lowercase on abcXYZ

### Subset the data

Select out the set of upper- and lowercase (a, b, c, x, y z, A, B, C, X, Y, Z). Note that some of these classes can be confusing (e.g., x and y).

### Train and tune models

Perform a full model training and hyperparameter tuning.

1. Select candidate models, hyperparameter options, and evaluation metric
2. Set aside a validation hold-out dataset
3. Train models over K splits (use k-fold or train/test split)
    1. Split train using k-fold with the number of folds equal to the number of parameter combinations
    2. Train on k-fold split
    3. Record performance of each set of parameters
    4. Use winning set of parameters to train model on full training set
    5. Record each model's performance on that split's test set
4. Evaluate model performance and promote one model as the winner
5. Train winning model on both train + test
6. Check model performance on the validation hold-out


## 4. (_Optional_) Model comparison: classify even vs odd

**NOTE:** This is a larger dataset (~400k rows) so it will require more memory and time to train models on it. 

Alternatively, you can train models on smaller subsets of the data to get a feel for which models perform better than others. Then train the winning model on the full dataset and validate against the hold-out.

### Subset the data

Select only digits and add a column for 'is_even'. Be sure to create a validation hold-out dataset for later.

### Build and compare models

Train at least two different models, compare the results and choose a winner based on an evaluation metric of your choice.

In [14]:
%pip install -q emnist pandas pyarrow numpy matplotlib seaborn scikit-learn xgboost tensorflow
%reset -f

Note: you may need to restart the kernel to use updated packages.


In [15]:
# Import packages
import os
import string
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import emnist
from IPython.display import display, Markdown

# ML packages
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
# Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
# XGBoost (SVM)
from xgboost import XGBClassifier
# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# Constants
SIZE = 28

In [16]:
# Define helper functions
def int_to_char(label):
    """Convert an integer label to the corresponding uppercase character."""
    if label < 10:
        return str(label)
    elif label < 36:
        return chr(label - 10 + ord('A'))
    else:
        return chr(label - 36 + ord('a'))

def show_image(row):
    """Display a single image and its corresponding label."""
    image = row['image']
    label = row['label']
    plt.imshow(image, cmap='gray')
    plt.title('Label: ' + int_to_char(label))
    plt.axis('off')
    plt.show()

def show_grid(data, title=None, num_cols=5, figsize=(20, 10)):
    """
    Display a list of images as a grid of num_cols columns.
    images: a list of images, each represented as a 28x28 numpy array
    labels: a list of labels, one for each image
    title: (optional) a title for the plot
    num_cols: (optional) number of columns to use in the grid
    figsize: (optional) size of the figure
    """
    num_images = len(data)
    num_rows = (num_images - 1) // num_cols + 1
    fig, axes = plt.subplots(num_rows, num_cols, figsize=figsize)
    if title is not None:
        fig.suptitle(title, fontsize=16)
    for i in range(num_rows):
        for j in range(num_cols):
            index = i * num_cols + j
            if index < num_images:
                axes[i, j].imshow(data.iloc[index]['image'], cmap='gray')
                axes[i, j].axis('off')
                label = int_to_char(data.iloc[index]['label'])
                axes[i, j].set_title(label)
    plt.show()

# Get a random image of a given label from the dataset
def get_image_by_label(data, label):
    """Get a random image of a given label from the dataset."""
    images = data[data['label'] == label]['image'].tolist()
    return random.choice(images)

# Plot the training and validation accuracy during the training of a model
def plot_accuracy(history):
    """Plot the training and validation accuracy during the training of a model."""
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    epochs = range(1, len(acc) + 1)
    plt.plot(epochs, acc, 'bo', label='Training accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

# Plot the training and validation loss during the training of a model
def plot_loss(history):
    """Plot the training and validation loss during the training of a model."""
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(loss) + 1)
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

# Normalize the pixel values of the images in the dataset to have zero mean and unit variance
# This is a common preprocessing step for neural networks, but may not be necessary in all cases
def normalize_images(images):
    """Normalize the pixel values of the images in the dataset to have zero mean and unit variance."""
    images = np.array(images)
    mean = images.mean()
    std = images.std()
    images = (images - mean) / std
    return images.tolist()

# Display metrics for a model
def display_metrics(task, model_name, metrics_dict):
    """Display performance metrics and confusion matrix for a model."""
    metrics_df = pd.DataFrame()
    cm_df = pd.DataFrame()
    for key, value in metrics_dict[task][model_name].items():
        if type(value) == np.ndarray:
            cm_df = pd.DataFrame(value, index=['actual 0', 'actual 1'], columns=['predicted 0', 'predicted 1'])
        else:
            metrics_df[key] = [value]
    display(Markdown(f'# Performance Metrics: {model_name}'))
    display(metrics_df)
    display(Markdown(f'# Confusion Matrix: {model_name}'))
    display(cm_df)

In [17]:
# Load data

# Extract the training split as images and labels
image, label = emnist.extract_training_samples('byclass')

# Add columns for each pixel value (28x28 = 784 columns)
emnist_train = pd.DataFrame()

# Add a column with the image data as a 28x28 array
emnist_train['image'] = list(image)
emnist_train['image_flat'] = emnist_train['image'].apply(lambda x: np.array(x).reshape(-1))

# Add a column showing the label
emnist_train['label'] = label

# Convert labels to characters
class_label = np.array([int_to_char(l) for l in label])

# Add a column with the character corresponding to the label
emnist_train['class'] = class_label

# Repeat for the test split
image, label = emnist.extract_test_samples('byclass')
class_label = np.array([int_to_char(l) for l in label])
emnist_test = pd.DataFrame()
emnist_test['image'] = list(image)
emnist_test['image_flat'] = emnist_test['image'].apply(lambda x: np.array(x).reshape(-1))
emnist_test['label'] = label
emnist_test['class'] = class_label

# Combine the training and test splits
emnist_all = pd.concat([emnist_train, emnist_test], axis=0)

# Subset for only digits 0-9
digits = emnist_all[emnist_all['label'] < 10]

# Subset for lowercase letters
lowercase = emnist_all[(emnist_all['class'] >= 'a') & (emnist_all['class'] <= 'z')]
uppercase = emnist_all[(emnist_all['class'] >= 'A') & (emnist_all['class'] <= 'Z')]

# Subset for upper- and lowercase letters a, b, c, d, e, f, g
a2g = emnist_all[(emnist_all['class'].isin(['a', 'b', 'c', 'd', 'e', 'f', 'g']))]

# Subset for upper- and lowercase letters a, b, c, x, y, z
abcxyz = emnist_all[(emnist_all['class'].isin(['a', 'b', 'c', 'A', 'B', 'C', \
                                               'x', 'y', 'z', 'X', 'Y', 'Z']))]

In [18]:
# Display the size of a2g, abcxyz, digits, and the full dataset
display(Markdown(f'# Dataset Sizes'))
display(Markdown(f'**a2g**: {len(a2g)}'))
display(Markdown(f'**abcxyz**: {len(abcxyz)}'))
display(Markdown(f'**digits**: {len(digits)}'))
display(Markdown(f'**emnist_all**: {len(emnist_all)}'))

# Dataset Sizes

**a2g**: 68795

**abcxyz**: 65926

**digits**: 402953

**emnist_all**: 814255

In [19]:
# FIXME: Classify lettters as uppercase/lowercase
abcxyz['is_upper'] = ...

# FIXME: Classify digits as even/odd
digits['is_even'] = ...

/var/folders/fg/nc6l7t216_z_mj4y85bhl4wc0000gn/T/ipykernel_44094/2067495769.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abcxyz['is_upper'] = ...
/var/folders/fg/nc6l7t216_z_mj4y85bhl4wc0000gn/T/ipykernel_44094/2067495769.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  digits['is_even'] = ...


Classify all symbols letters a -> g

In [23]:
#Split back to train and test data
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(a2g['image_flat'].tolist(), a2g['class'], test_size=0.3, random_state=42)

In [24]:
# Choose random forest
from scipy.stats import randint

# Define the parameter distribution
param_dist = {
    'n_estimators': randint(10, 50),  # Example: Number of trees in a range
    'max_depth': [3, 5, 7, 10],  # Example: Maximum depth of the tree
    # Add more parameters and distributions here
}

rf_a2g = RandomForestClassifier()

from sklearn.model_selection import RandomizedSearchCV
# Initialize the RandomizedSearchCV object
random_search = RandomizedSearchCV(estimator=rf_a2g, param_distributions=param_dist, n_iter=50, cv=3, n_jobs=-1, verbose=2, random_state=42)
#If you don't explicitly specify the scoring parameter, it defaults to the estimator's default scorer (if available), which, for most classifiers, is accuracy.
#Reducing number of iterations and CV to make sure run time is not long

# Split the data into training and test sets
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

# Fit the random search to the data
random_search.fit(X_train2, y_train2)

# Print the best parameters
print("Best parameters found: ", random_search.best_params_)

# Use the best estimator for further predictions
best_rf_a2g = random_search.best_estimator_

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] END .......................max_depth=7, n_estimators=17; total time=   1.9s
[CV] END .......................max_depth=7, n_estimators=17; total time=   2.1s
[CV] END .......................max_depth=7, n_estimators=38; total time=   3.6s
[CV] END .......................max_depth=7, n_estimators=17; total time=   2.0s
[CV] END .......................max_depth=7, n_estimators=38; total time=   3.9s
[CV] END .......................max_depth=3, n_estimators=30; total time=   1.9s
[CV] END .......................max_depth=7, n_estimators=38; total time=   4.0s
[CV] END .......................max_depth=3, n_estimators=30; total time=   1.5s
[CV] END .......................max_depth=3, n_estimators=30; total time=   1.4s
[CV] END .......................max_depth=7, n_estimators=28; total time=   2.4s
[CV] END .......................max_depth=7, n_estimators=28; total time=   2.3s
[CV] END .......................max_depth=7, n_

In [25]:
n_est = random_search.best_params_['n_estimators']
md = random_search.best_params_['max_depth']
rf_a2g_best = RandomForestClassifier(n_estimators= n_est, max_depth = md)

# Train the model here
rf_a2g_best.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, n_estimators=49)

In [32]:
from sklearn.metrics import accuracy_score

# Assuming pred contains the predictions from your model
# and y_val contains the true labels
pred = rf_a2g_best.predict(X_val)
accuracy_rf_a2g = accuracy_score(y_val, pred)
print(f'The total accuracy_score is {accuracy_rf_a2g}')

The total accuracy_score is 0.942584427540094


In [33]:
from sklearn.metrics import classification_report

# Assuming y_val contains the true labels and pred contains your model predictions
report = classification_report(y_val, pred, output_dict=True)
print(classification_report(y_val, pred))

# If you want to access per-class scores programmatically:
for class_name, metrics in report.items():
    if class_name not in ('accuracy', 'macro avg', 'weighted avg'):
        print(f"Class {class_name} - Precision: {metrics['precision']:.2f}, Recall: {metrics['recall']:.2f}, F1 Score: {metrics['f1-score']:.2f}")


              precision    recall  f1-score   support

           a       0.91      0.95      0.93      3615
           b       0.97      0.94      0.95      1800
           c       0.93      0.70      0.80       954
           d       0.95      0.97      0.96      3513
           e       0.95      0.98      0.96      8598
           f       0.94      0.87      0.91       894
           g       0.94      0.83      0.88      1265

    accuracy                           0.94     20639
   macro avg       0.94      0.89      0.91     20639
weighted avg       0.94      0.94      0.94     20639

Class a - Precision: 0.91, Recall: 0.95, F1 Score: 0.93
Class b - Precision: 0.97, Recall: 0.94, F1 Score: 0.95
Class c - Precision: 0.93, Recall: 0.70, F1 Score: 0.80
Class d - Precision: 0.95, Recall: 0.97, F1 Score: 0.96
Class e - Precision: 0.95, Recall: 0.98, F1 Score: 0.96
Class f - Precision: 0.94, Recall: 0.87, F1 Score: 0.91
Class g - Precision: 0.94, Recall: 0.83, F1 Score: 0.88


It seems that the model is more struggling at classifying c
It was noticible that c has a much lower recall (i.e. positive predicted values) - 30% of the time when it came across c, it gave a different value instead of c
That lead to a low F1 score of C

Perhaps adding more data on C, or upweight the value of c may help with the F1 score